In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4

cuda


In [2]:
with open('TGG.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\t', '\n', ' ', '!', '$', '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ç', 'é', 'ê', 'ô', '\u200a', '—', '‘', '’', '“', '”', '…']


In [3]:
string_to_int = { ch: i for i, ch in enumerate(chars) }
int_to_string = { i: ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([ 0,  0,  0,  2,  2,  2, 43, 59, 56,  2, 30, 69, 56, 52, 71,  2, 30, 52,
        71, 70, 53, 76,  1,  0,  0,  0,  0,  2,  2, 53, 76,  1,  0,  0,  0,  2,
        29, 10,  2, 42, 54, 66, 71, 71,  2, 29, 60, 71, 77, 58, 56, 69, 52, 63,
        55,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 43, 52, 53, 63, 56,
         2, 66, 57,  2, 26, 66, 65, 71, 56, 65])


In [4]:
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    print(ix)
    x = torch.stack([data[i: i + block_size] for i in ix])
    y = torch.stack([data[i + 1: i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('Inputs: ')
print(x)
print('Targets: ')
print(y)
    

tensor([ 30725, 195678, 214097,  90040])
Inputs: 
tensor([[ 1, 70, 60, 55, 56,  2, 53, 76],
        [ 2, 72, 67,  2, 66, 65,  2, 59],
        [60, 65, 58,  2, 71, 66,  2, 71],
        [85, 70,  2, 52,  2, 53, 66, 66]], device='cuda:0')
Targets: 
tensor([[70, 60, 55, 56,  2, 53, 76,  2],
        [72, 67,  2, 66, 65,  2, 59, 66],
        [65, 58,  2, 71, 66,  2, 71, 72],
        [70,  2, 52,  2, 53, 66, 66, 71]], device='cuda:0')


In [9]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)

            logits = logits[:, -1, :]

            probs = F.softmax(logits, dim = -1)
            index_next = torch.multinomial(probs, num_samples = 1)
            index = torch.cat((index, index_next), dim = 1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype = torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

	Hdx9eT[ GS(p1LzNTFZX”…“UlX7d K4scMgê“2—pNéHWz!*	I…]jSS”c:1Kêsqç’Uyq”dw—J?c5$$D“c“…S 4sMLLô1*ObbO’’]Gç…[;R,EL]V!“w1S;kULt*Rp:ç	fbSq;.2Zj(tcw$v  l]êJêmd$SVxDEx—)pUaIs2ceqô1sbvêv	MzQBU-zv
Eif?B0Iv!))kB aj-JdK8zçç?Tê5$’q?dAYggJ5tn‘2sV5$é5[hMjUNmc!d4ggBQQ? fZR15I )z?j“wrAO9Gje7L;zoi;”J5H:y7s)br[]BM 	6TsQz4Zx!Jb0l[E	UOVe—DkuLzuM.nw*“2RSJi;UD;ô:G‘F1D4gBX’psDi:.Lu6 FZç?…]jô5ô*PPdk 50CLHVZLgêC !C5fy cr,wvD)0j2hf	YIe“Y(o)-mxeét)!J5R’miGSVX7c5nlO’mR8phXN d8—:,çELUy3	U33êôF].OL”4SVhQFaO9aHp8oO8m]54.!êL!?9*y
